In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()

In [2]:
slice = fablib.get_slice(name="hadoop-cluster-icdm")

In [3]:
server_details = []
for node in slice.get_nodes():
    dtl = {}
    if node.get_name()[:3] == 'svr':
        dtl['name'] = node.get_name()
        dtl['addr'] = node.get_interfaces()[0].get_ip_addr_ssh()
        dtl['ssh'] = node.get_ssh_command()
        server_details.append(dtl)

In [4]:
cluster_size = len(server_details)

In [5]:
with open('./config_files/hadoop_hosts', 'w') as f:
    for i in range(cluster_size):
        f.write(f'10.{i}.0.2 svr{i}\n')

In [6]:
name_node = slice.get_node(name='svr0')

In [7]:
name_node.execute("ssh-keygen -t rsa -P '' -f ~/.ssh/id_rsa")

Generating public/private rsa key pair.
Your identification has been saved in /home/ubuntu/.ssh/id_rsa
Your public key has been saved in /home/ubuntu/.ssh/id_rsa.pub
The key fingerprint is:
SHA256:LPCUt8MPRNdEG/EV9ebQaRnMw2EB6nSajtEpaYtUqLM ubuntu@svr0
The key's randomart image is:
+---[RSA 3072]----+
|        . .+=o=*B|
|       o..  o+.B=|
|    . o.o. o..o+=|
|     +.=..= = .+ |
|     oo.S= *    .|
|      +.o+*      |
|     E . o..     |
|                 |
|                 |
+----[SHA256]-----+


("Generating public/private rsa key pair.\nYour identification has been saved in /home/ubuntu/.ssh/id_rsa\nYour public key has been saved in /home/ubuntu/.ssh/id_rsa.pub\nThe key fingerprint is:\nSHA256:LPCUt8MPRNdEG/EV9ebQaRnMw2EB6nSajtEpaYtUqLM ubuntu@svr0\nThe key's randomart image is:\n+---[RSA 3072]----+\n|        . .+=o=*B|\n|       o..  o+.B=|\n|    . o.o. o..o+=|\n|     +.=..= = .+ |\n|     oo.S= *    .|\n|      +.o+*      |\n|     E . o..     |\n|                 |\n|                 |\n+----[SHA256]-----+\n",
 '')

In [8]:
name_node.download_file("./config_files/id_rsa.pub", "/home/ubuntu/.ssh/id_rsa.pub")

In [9]:
with open('./config_files/ssh_config', 'w') as f:
    for i in range(cluster_size):
        f.write(f'Host svr{i}\n')
        f.write(f'  HostName 10.{i}.0.2\n')
        f.write(f'  User ubuntu\n')
        f.write(f'  IdentityFile ~/.ssh/id_rsa\n')

In [10]:
with open('./config_files/hadoop_config_files/workers', 'w') as f:
    for i in range(1, cluster_size):
        f.write(f'10.{i}.0.2\n')

In [11]:
for node in slice.get_nodes():
    node_name = node.get_name()
    if node_name[:3] == 'svr':
        node.execute("sudo apt-get update -y", quiet=True)
        node.execute("mkdir /home/ubuntu/hadoop_setup_files")
        node.upload_file("./config_files/hadoop_hosts", "/home/ubuntu/hadoop_setup_files/hadoop_hosts")
        node.execute("sudo cp /home/ubuntu/hadoop_setup_files/hadoop_hosts /etc/hosts")
        
        if node_name != 'svr0':
            node.upload_file("./config_files/id_rsa.pub", "/home/ubuntu/.ssh/id_rsa.pub")
        if node_name == 'svr0':
            node.upload_file("./config_files/ssh_config", "/home/ubuntu/.ssh/config")
        node.execute("cat ~/.ssh/id_rsa.pub >> ~/.ssh/authorized_keys")
        
        node.execute("sudo apt-get -y install openjdk-8-jdk-headless", quiet=True)
        node.execute("wget https://dlcdn.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz", quiet=True)
        node.execute("tar xvzf hadoop-3.3.6.tar.gz", quiet=True)
        node.execute("mv hadoop-3.3.6 hadoop", quiet=True)
        node.execute("sudo mkdir -p /usr/local/hadoop/hdfs/namenode/data", quiet=True)
        node.execute("sudo mkdir -p /usr/local/hadoop/hdfs/datanode/data", quiet=True)
        node.execute("sudo chown -R ubuntu:ubuntu /usr/local/hadoop/hdfs/namenode/data", quiet=True)
        node.execute("sudo chown -R ubuntu:ubuntu /usr/local/hadoop/hdfs/datanode/data", quiet=True)

In [12]:
for node in slice.get_nodes():
    node_name = node.get_name()
    if node_name[:3] == 'svr':
        
        node.upload_file("./config_files/hadoop_config_files/bkup-hadoop-env.sh",   "/home/ubuntu/hadoop_setup_files/hadoop-env.sh")
        node.upload_file("./config_files/hadoop_config_files/bkup-core-site.xml",   "/home/ubuntu/hadoop_setup_files/core-site.xml")
        node.upload_file("./config_files/hadoop_config_files/bkup-hdfs-site.xml",   "/home/ubuntu/hadoop_setup_files/hdfs-site.xml")
        node.upload_file("./config_files/hadoop_config_files/bkup-yarn-site.xml",   "/home/ubuntu/hadoop_setup_files/yarn-site.xml")
        node.upload_file("./config_files/hadoop_config_files/bkup-mapred-site.xml", "/home/ubuntu/hadoop_setup_files/mapred-site.xml")
        node.upload_file("./config_files/hadoop_config_files/workers",              "/home/ubuntu/hadoop_setup_files/workers")
        node.upload_file("./config_files/bashrc_update",                            "/home/ubuntu/hadoop_setup_files/bashrc_update")
        
        node.execute("sudo cp /home/ubuntu/hadoop_setup_files/hadoop-env.sh    /home/ubuntu/hadoop/etc/hadoop/hadoop-env.sh")
        node.execute("sudo cp /home/ubuntu/hadoop_setup_files/core-site.xml    /home/ubuntu/hadoop/etc/hadoop/core-site.xml")
        node.execute("sudo cp /home/ubuntu/hadoop_setup_files/hdfs-site.xml    /home/ubuntu/hadoop/etc/hadoop/hdfs-site.xml")
        node.execute("sudo cp /home/ubuntu/hadoop_setup_files/yarn-site.xml    /home/ubuntu/hadoop/etc/hadoop/yarn-site.xml")
        node.execute("sudo cp /home/ubuntu/hadoop_setup_files/mapred-site.xml  /home/ubuntu/hadoop/etc/hadoop/mapred-site.xml")
        node.execute("sudo cp /home/ubuntu/hadoop_setup_files/workers          /home/ubuntu/hadoop/etc/hadoop/workers")
        node.execute("cat /home/ubuntu/hadoop_setup_files/bashrc_update >> /home/ubuntu/.bashrc")
        node.execute("source /home/ubuntu/.bashrc")

In [13]:
server_details

[{'name': 'svr0',
  'addr': '10.0.0.2',
  'ssh': 'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:feb5:539c'},
 {'name': 'svr1',
  'addr': '10.1.0.2',
  'ssh': 'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fe56:5a56'},
 {'name': 'svr2',
  'addr': '10.2.0.2',
  'ssh': 'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fe74:a331'}]

In [14]:
name_node.execute("/home/ubuntu/hadoop/bin/hdfs namenode -format")

2023-12-08 00:43:03,353 INFO namenode.NameNode: STARTUP_MSG: 
/************************************************************
STARTUP_MSG: Starting NameNode
STARTUP_MSG:   host = svr0/10.0.0.2
STARTUP_MSG:   args = [-format]
STARTUP_MSG:   version = 3.3.6
STARTUP_MSG:   classpath = /home/ubuntu/hadoop/etc/hadoop:/home/ubuntu/hadoop/share/hadoop/common/lib/netty-transport-native-epoll-4.1.89.Final-linux-aarch_64.jar:/home/ubuntu/hadoop/share/hadoop/common/lib/jakarta.activation-api-1.2.1.jar:/home/ubuntu/hadoop/share/hadoop/common/lib/curator-client-5.2.0.jar:/home/ubuntu/hadoop/share/hadoop/common/lib/animal-sniffer-annotations-1.17.jar:/home/ubuntu/hadoop/share/hadoop/common/lib/httpclient-4.5.13.jar:/home/ubuntu/hadoop/share/hadoop/common/lib/jackson-mapper-asl-1.9.13.jar:/home/ubuntu/hadoop/share/hadoop/common/lib/netty-transport-native-epoll-4.1.89.Final-linux-x86_64.jar:/home/ubuntu/hadoop/share/hadoop/common/lib/netty-codec-socks-4.1.89.Final.jar:/home/ubuntu/hadoop/share/hadoop/co

('',
 "WARNING: /home/ubuntu/hadoop/logs does not exist. Creating.\n2023-12-08 00:43:03,353 INFO namenode.NameNode: STARTUP_MSG: \n/************************************************************\nSTARTUP_MSG: Starting NameNode\nSTARTUP_MSG:   host = svr0/10.0.0.2\nSTARTUP_MSG:   args = [-format]\nSTARTUP_MSG:   version = 3.3.6\nSTARTUP_MSG:   classpath = /home/ubuntu/hadoop/etc/hadoop:/home/ubuntu/hadoop/share/hadoop/common/lib/netty-transport-native-epoll-4.1.89.Final-linux-aarch_64.jar:/home/ubuntu/hadoop/share/hadoop/common/lib/jakarta.activation-api-1.2.1.jar:/home/ubuntu/hadoop/share/hadoop/common/lib/curator-client-5.2.0.jar:/home/ubuntu/hadoop/share/hadoop/common/lib/animal-sniffer-annotations-1.17.jar:/home/ubuntu/hadoop/share/hadoop/common/lib/httpclient-4.5.13.jar:/home/ubuntu/hadoop/share/hadoop/common/lib/jackson-mapper-asl-1.9.13.jar:/home/ubuntu/hadoop/share/hadoop/common/lib/netty-transport-native-epoll-4.1.89.Final-linux-x86_64.jar:/home/ubuntu/hadoop/share/hadoop/common/

In [15]:
name_node.execute("/home/ubuntu/hadoop/sbin/start-dfs.sh")

Starting namenodes on [svr0]
svr0: Warning: Permanently added '10.0.0.2' (ECDSA) to the list of known hosts.
Starting datanodes
10.2.0.2: Warning: Permanently added '10.2.0.2' (ECDSA) to the list of known hosts.
10.2.0.2: WARNING: /home/ubuntu/hadoop/logs does not exist. Creating.
10.1.0.2: Warning: Permanently added '10.1.0.2' (ECDSA) to the list of known hosts.
10.1.0.2: WARNING: /home/ubuntu/hadoop/logs does not exist. Creating.
Starting secondary namenodes [svr0]


("Starting namenodes on [svr0]\nsvr0: Warning: Permanently added '10.0.0.2' (ECDSA) to the list of known hosts.\r\nStarting datanodes\n10.2.0.2: Warning: Permanently added '10.2.0.2' (ECDSA) to the list of known hosts.\r\n10.2.0.2: WARNING: /home/ubuntu/hadoop/logs does not exist. Creating.\n10.1.0.2: Warning: Permanently added '10.1.0.2' (ECDSA) to the list of known hosts.\r\n10.1.0.2: WARNING: /home/ubuntu/hadoop/logs does not exist. Creating.\nStarting secondary namenodes [svr0]\n",
 '')

In [16]:
name_node.execute("/home/ubuntu/hadoop/sbin/start-yarn.sh")

Starting resourcemanager
Starting nodemanagers


('Starting resourcemanager\nStarting nodemanagers\n', '')